- add simple padding when dimension of input and output is different
- https://www.kaggle.com/meaninglesslives/using-decision-trees-for-arc
- https://www.kaggle.com/davidbnn92/task-tagging
- https://www.kaggle.com/nxrprime/grid-search-with-xgboost-and-cv

In [1]:
import os
import json
import numpy as np
import pandas as pd
from math import floor
from pathlib import Path
from itertools import product
from matplotlib import colors
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from itertools import permutations
from lightgbm import LGBMClassifier
from collections import defaultdict
from catboost import CatBoostClassifier
from sklearn.ensemble import BaggingClassifier
from itertools import combinations,permutations
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [2]:
data_path = Path('/kaggle/input/abstraction-and-reasoning-challenge/')
training_path = data_path / 'training'
evaluation_path = data_path / 'evaluation'
test_path = data_path / 'test'

In [3]:
def preprocess(task, index): # modify mistakes in data
    # 025d127b
    if index == "025d127b.json":
        for i in range(9, 12):
            for j in range(3, 8):
                task['train'][0]['output'][i][j] = 0
        for i in range(7, 10):
            for j in range(3, 6):
                task['train'][0]['output'][i][j] = 2
        task['train'][0]['output'][8][4] = 0
    # ef135b50
    elif index == "ef135b50.json":
        task['test'][0]['output'][6][4] = 9
    # bd14c3bf
    elif index == "bd14c3bf.json":
        for i in range(3):
            for j in range(5):
                if task['test'][0]['input'][i][j] == 1:
                    task['test'][0]['input'][i][j] = 2
    # a8610ef7
    elif index == "a8610ef7.json":
        for i in range(6):
            for j in range(6):
                if task['test'][0]['output'][i][j] == 8:
                    task['test'][0]['output'][i][j] = 5
        task['train'][3]['input'][0][1] = 2
        task['train'][3]['input'][5][1] = 2
    # 54db823b
    elif index == "54db823b.json":
        task['train'][0]['output'][2][3] = 3
        task['train'][0]['output'][2][4] = 9
    # e5062a87
    elif index == "e5062a87.json":
        for j in range(3, 7):
            task['train'][1]['output'][1][j] = 2
    # 1b60fb0c
    elif index == "1b60fb0c.json":
        task['train'][1]['output'][8][8] = 0
        task['train'][1]['output'][8][9] = 0
    # 82819916
    elif index == "82819916.json":
        task['train'][0]['output'][4][5] = 4
    # fea12743
    elif index == "fea12743.json":
        for i in range(11, 16):
            for j in range(6):
                if task['train'][0]['output'][i][j] == 2:
                    task['train'][0]['output'][i][j] = 8
    # 42a50994
    elif index == "42a50994.json":
        task['train'][0]['output'][1][0] = 8
        task['train'][0]['output'][0][1] = 8
    # f8be4b64
    elif index == "f8be4b64.json":
        for j in range(19):
            if task['test'][0]['output'][12][j] == 0:
                task['test'][0]['output'][12][j] = 1
        task['test'][0]['output'][12][8] = 0
    # d511f180
    elif index == "d511f180.json":
        task['train'][1]['output'][2][2] = 9
    # 10fcaaa3
    elif index == "10fcaaa3.json":
        task['train'][1]['output'][4][7] = 8
    # cbded52d
    elif index == "cbded52d.json":
        task['train'][0]['input'][4][6] = 1
    # 11852cab
    elif index == "11852cab.json":
        task['train'][0]['input'][1][2] = 3
    # 868de0fa
    elif index == "868de0fa.json":
        for j in range(2, 9):
            task['train'][2]['input'][9][j] = 0
            task['train'][2]['input'][10][j] = 1
            task['train'][2]['input'][15][j] = 0
            task['train'][2]['input'][16][j] = 1
        task['train'][2]['input'][15][2] = 1
        task['train'][2]['input'][15][8] = 1
    # 6d58a25d
    elif index == "6d58a25d.json":
        task['train'][0]['output'][10][0] = 0
        task['train'][2]['output'][6][13] = 4
    # a9f96cdd
    elif index == "a9f96cdd.json":
        task['train'][3]['output'][1][3] = 0
    # 48131b3c
    elif index == "48131b3c.json":
        task['train'][2]['output'][4][4] = 0
    # 150deff5
    elif index == "150deff5.json":
        aux = task['train'][2]['output'].copy()
        task['train'][2]['output'] = task['train'][2]['input'].copy()
        task['train'][2]['input'] = aux
    # 17cae0c1
    elif index == "17cae0c1.json":
        for i in range(3):
            for j in range(3, 6):
                task['test'][0]['output'][i][j] = 9
    # e48d4e1a
    elif index == "e48d4e1a.json":
        task['train'][3]['input'][0][9] = 5
        task['train'][3]['output'][0][9] = 0
    # 8fbca751
    elif index == "8fbca751.json":
        task['train'][1]['output'][1][3] = 2
        task['train'][1]['output'][2][3] = 8
    # 4938f0c2
    elif index == "4938f0c2.json":
        for i in range(12):
            for j in range(6,13):
                if task['train'][2]['input'][i][j]==2:
                    task['train'][2]['input'][i][j] = 0
        for i in range(5,11):
            for j in range(7):
                if task['train'][2]['input'][i][j]==2:
                    task['train'][2]['input'][i][j] = 0
    # 9aec4887
    elif index == "9aec4887.json":
        task['train'][0]['output'][1][4] = 8
    # b0f4d537
    elif index == "b0f4d537.json":
        for i in range(9):
            task['train'][0]['output'][i][3] = 0
            task['train'][0]['output'][i][4] = 1
        task['train'][0]['output'][2][3] = 3
        task['train'][0]['output'][2][4] = 3
        task['train'][0]['output'][5][3] = 2
    # aa300dc3
    elif index == "aa300dc3.json":
        task['train'][1]['input'][1][7] = 5
        task['train'][1]['output'][1][7] = 5
        task['train'][1]['input'][8][2] = 5
        task['train'][1]['output'][8][2] = 5
    # ad7e01d0
    elif index == "ad7e01d0.json":
        task['train'][0]['output'][6][7] = 0
    # a8610ef7
    elif index == "a8610ef7.json":
        task['train'][3]['input'][0][1] = 0
        task['train'][3]['input'][5][1] = 0
        task['train'][3]['output'][0][1] = 0
        task['train'][3]['output'][5][1] = 0
    # 97239e3d
    elif index == "97239e3d.json":
        task['test'][0]['input'][14][6] = 0
        task['test'][0]['input'][14][10] = 0
    # d687bc17
    elif index == "d687bc17.json":
        task['train'][2]['output'][7][1] = 4
    return task

In [4]:
def flattener(pred):
    str_pred = str([row for row in pred])
    str_pred = str_pred.replace(', ', '')
    str_pred = str_pred.replace('[[', '|')
    str_pred = str_pred.replace('][', '|')
    str_pred = str_pred.replace(']]', '|')
    return str_pred

# task tagging

In [5]:
def color_check(color_list):
    tmp = color_list[0]
    for i in range(1,len(color_list)):
        if set(tmp) != set(color_list[i]):
            return False
    return True

def create_df(folder_path):
    task_names_list = sorted(os.listdir(folder_path))
    task_list = []
    for task_name in task_names_list: 
        task_file = str(folder_path / task_name)
        with open(task_file, 'r') as f:
            task = json.load(f)
            if "test" not in str(folder_path):
                task = preprocess(task, task_name)
            task_list.append(task)
    
    df = pd.DataFrame()
    df['task_name'] = task_names_list
    df['task'] = task_list
    df['number_of_train_pairs'] = df['task'].apply(lambda x: len(x['train']))
    df['number_of_test_pairs'] = df['task'].apply(lambda x: len(x['test']))
    
    # Compare image sizes
    df['inputs_all_have_same_height'] = df['task'].apply(
        lambda task: int(len(set([len(example['input']) for example in task['train']])) == 1)
    )
    df['inputs_all_have_same_width'] = df['task'].apply(
        lambda task: int(len(set([len(example['input'][0]) for example in task['train']])) == 1)
    )
    df['inputs_all_have_same_shape'] = df['inputs_all_have_same_height'] * df['inputs_all_have_same_width']
    df['input_height_if_constant'] = df['task'].apply(
        lambda task: len(task['train'][0]['input'])
                     if (len(set([len(example['input']) for example in task['train']])) == 1)
                     else np.nan
    )
    df['input_width_if_constant'] = df['task'].apply(
        lambda task: len(task['train'][0]['input'][0])
                     if (len(set([len(example['input'][0]) for example in task['train']])) == 1)
                     else np.nan
    )
    df['outputs_all_have_same_height'] = df['task'].apply(
        lambda task: int(len(set([len(example['output']) for example in task['train']])) == 1)
    )
    df['outputs_all_have_same_width'] = df['task'].apply(
        lambda task: int(len(set([len(example['output'][0]) for example in task['train']])) == 1)
    )
    df['outputs_all_have_same_shape'] = df['outputs_all_have_same_height'] * df['outputs_all_have_same_width']
    df['output_height_if_constant'] = df['task'].apply(
        lambda task: len(task['train'][0]['output'])
                     if (len(set([len(example['output']) for example in task['train']])) == 1)
                     else np.nan
    )
    df['output_width_if_constant'] = df['task'].apply(
        lambda task: len(task['train'][0]['output'][0])
                     if (len(set([len(example['output'][0]) for example in task['train']])) == 1)
                     else np.nan
    )  
    df['in_each_pair_shape_doesnt_change'] = df['task'].apply(
        lambda task: np.prod([int(len(example['input'][0])==len(example['output'][0])
                                  and len(example['input'])==len(example['output'])
                                 ) for example in task['train']
                            ])
    )
    df['in_each_pair_shape_ratio_is_the_same'] = df['task'].apply(
        lambda task: (len(set([len(example['input'][0]) / len(example['output'][0])
                                 for example in task['train']]))==1) * (
                      len(set([len(example['input']) / len(example['output'])
                                 for example in task['train']]))==1)
    )
    df['o/i_height_ratio_if_constant'] = df['task'].apply(
        lambda task: len(task['train'][0]['output']) / len(task['train'][0]['input'])
                     if (len(set([len(example['input']) / len(example['output'])
                                 for example in task['train']]))==1)
                     else np.nan
    )
    df['o/i_width_ratio_if_constant'] = df['task'].apply(
        lambda task: len(task['train'][0]['output'][0]) / len(task['train'][0]['input'][0])
                     if (len(set([len(example['input'][0]) / len(example['output'][0])
                                 for example in task['train']]))==1)
                     else np.nan
    )
    
    # my idea ---------
    df["same_color_sum"] = df['task'].apply(lambda task: 
                        np.all([int(sum(sum(np.array(example['input'])))== sum(sum(np.array(example['output'])))) for example in task['train']]))
    
    df["same_color_sum_in_edge"] = df['task'].apply(lambda task: 
                        np.all([int(sum(np.array(example['input'])[0,:]) +sum(np.array(example['input'])[:,0]) + 
                                    sum(np.array(example['input'])[-1,:]) +sum(np.array(example['input'])[:,-1])
                                    == 
                                    sum(np.array(example['output'])[0,:]) +sum(np.array(example['output'])[:,0]) + 
                                    sum(np.array(example['output'])[-1,:]) +sum(np.array(example['output'])[:,-1])) for example in task['train']]))
    
    df["io_color_kind_diff"] = df['task'].apply(lambda task: [len(np.unique(np.array(example['input']))) - len(np.unique(np.array(example['output']))) for example in task['train']])
    df["io_color_kind_diff_constant"] = df['io_color_kind_diff'].apply(lambda task: np.unique(np.array(task))[0] if len(np.unique(np.array(task)))==1 else -1)
    df["output_not_include_0"] = df['task'].apply(lambda task: np.all([np.all(np.array(example['output']) > 0) for example in task['train']]))
    df["increase_color_sum"] = df['task'].apply(lambda task: 
                        np.all([int(sum(sum(np.array(example['input']))) < sum(sum(np.array(example['output'])))) for example in task['train']]))
    df["decrease_color_sum"] = df['task'].apply(lambda task: 
                        np.all([int(sum(sum(np.array(example['input']))) > sum(sum(np.array(example['output'])))) for example in task['train']]))
    # **
    df["input_color_change_or_not"] = df['task'].apply(lambda task: color_check([list(np.unique(np.array(example['input']))) for example in task["train"]] ))
    # **
    df['color_kind_increase'] = df['task'].apply(
        lambda task: np.all([len(np.unique(np.array(example['input']))) < len(np.unique(np.array(example['output']))) for example in task['train']]))
    df['color_kind_decrease'] = df['task'].apply(
        lambda task: np.all([len(np.unique(np.array(example['input']))) > len(np.unique(np.array(example['output']))) for example in task['train']]))
    df['smaller_output'] = df['task'].apply(
        lambda task: np.all([(np.array(example['input']).shape[0] > np.array(example['output']).shape[0] or 
                             np.array(example['input']).shape[1] > np.array(example['output']).shape[1]) for example in task['train']]))
    return df

training_descriptive_df = create_df(training_path)
evaluation_descriptive_df = create_df(evaluation_path)
test_descriptive_df = create_df(test_path)

In [6]:
def classification(row):
    # same shape and color doesn't change in input and color kind decrease
    if row["in_each_pair_shape_doesnt_change"] == 1 and row["o/i_height_ratio_if_constant"] ==1 and row["o/i_width_ratio_if_constant"]==1 and row.input_color_change_or_not==1 and row.color_kind_decrease==1:
        return 1
    # same shape and color doesn't change in input and color kind increase
    elif row["in_each_pair_shape_doesnt_change"] == 1 and row["o/i_height_ratio_if_constant"] ==1 and row["o/i_width_ratio_if_constant"]==1 and row.input_color_change_or_not==1 and row.color_kind_increase==1:
        return 2
    # same shape and color doesn't change in input and color kind same
    elif row["in_each_pair_shape_doesnt_change"] == 1 and row["o/i_height_ratio_if_constant"] ==1 and row["o/i_width_ratio_if_constant"]==1 and row.input_color_change_or_not==1 and row.color_kind_increase==0  and row.color_kind_decrease==0:
        return 3
    # same shape and decrease color sum → xgboost
    if row["in_each_pair_shape_doesnt_change"] == 1 and row["o/i_height_ratio_if_constant"] ==1 and row["o/i_width_ratio_if_constant"]==1 and row.input_color_change_or_not==0 and row.color_kind_decrease==1:
        return 4
    # different shape and decrease color sum
    if row["in_each_pair_shape_doesnt_change"] == 1 and row["o/i_height_ratio_if_constant"] ==1 and row["o/i_width_ratio_if_constant"]==1 and row.input_color_change_or_not==0 and row.color_kind_increase==1:
        return 5
    # different shape and increase color sum
    elif row["in_each_pair_shape_doesnt_change"] == 1 and row["o/i_height_ratio_if_constant"] ==1 and row["o/i_width_ratio_if_constant"]==1 and row.input_color_change_or_not==0 and row.color_kind_increase==0  and row.color_kind_decrease==0:
        return 6
    # otherwise
    elif row["smaller_output"] == 1 and row["outputs_all_have_same_shape"] == 1:
        return 7
    else:
        return 8

training_descriptive_df["class"] = training_descriptive_df.apply(lambda x: classification(x), axis=1)
evaluation_descriptive_df["class"] = evaluation_descriptive_df.apply(lambda x: classification(x), axis=1)
test_descriptive_df["class"] = test_descriptive_df.apply(lambda x: classification(x), axis=1)

In [7]:
#print(training_descriptive_df["class"].value_counts(normalize=True))
#print(evaluation_descriptive_df["class"].value_counts(normalize=True))
#print(test_descriptive_df["class"].value_counts(normalize=True))

# feature engineering

In [8]:
def neighbours(color, cur_row, cur_col, nrows, ncols, radius):

    if cur_row<=radius-1: top = -1
    else: top = color[cur_row-radius][cur_col]
        
    if cur_row>=nrows-radius: bottom = -1
    else: bottom = color[cur_row+radius][cur_col]
        
    if cur_col<=radius-1: left = -1
    else: left = color[cur_row][cur_col-radius]
        
    if cur_col>=ncols-radius: right = -1
    else: right = color[cur_row][cur_col+radius]
        
    return top, bottom, left, right

def get_tl_tr(color, cur_row, cur_col, nrows, ncols, radius):
        
    if cur_row<=radius-1:
        top_left = -1
        top_right = -1
    else:
        if cur_col<=radius-1: top_left=-1
        else: top_left = color[cur_row-radius][cur_col-radius]
        if cur_col>=ncols-radius: top_right=-1
        else: top_right = color[cur_row-radius][cur_col+radius]   
        
    return top_left, top_right

def get_bl_br(color, cur_row, cur_col, nrows, ncols, radius):
        
    if cur_row>=nrows-radius:
        bottom_left = -1
        bottom_right = -1
    else:
        if cur_col<=radius-1: bottom_left=-1
        else: bottom_left = color[cur_row+radius][cur_col-radius]
        if cur_col>=ncols-radius: bottom_right=-1
        else: bottom_right = color[cur_row+radius][cur_col+radius]   
        
    return bottom_left, bottom_right

def diagonal(color, cur_row, cur_col, nrows, ncols, direction):
    element = []
    element.append(color[cur_row, cur_col])
    if direction == "upper-right":
        for i in range(-nrows,nrows):
            if (cur_row + i < nrows and cur_row +i >=0) and (cur_col - i < ncols and cur_col - i >=0):
                element.append(color[cur_row+i][cur_col-i])
            else:
                continue
    else:
        for i in range(-nrows,nrows):  
            if (cur_row + i < nrows and cur_row +i >=0) and (cur_col + i < ncols and cur_col + i >=0):
                element.append(color[cur_row+i][cur_col+i])
            else:
                continue
        
    return np.array(element)

def make_features(input_color, nfeat):
    nrows, ncols = input_color.shape
    feat = np.zeros((nrows*ncols,nfeat))
    cur_idx = 0
    for i in range(nrows):
        for j in range(ncols):
            feat[cur_idx,0] = i
            feat[cur_idx,1] = j
            feat[cur_idx,2] = input_color[i][j]
            feat[cur_idx,3:7] = neighbours(input_color, i, j, nrows, ncols,1)
            feat[cur_idx,7:9] = get_tl_tr(input_color, i, j, nrows, ncols,1)
            feat[cur_idx,9] = len(np.unique(input_color[i,:]))
            feat[cur_idx,10] = len(np.unique(input_color[:,j]))
            feat[cur_idx,11] = (i+j)
            feat[cur_idx,12] = len(np.unique(input_color[i-1:i+1,j-1:j+1]))
            feat[cur_idx,13:15] = get_bl_br(input_color, i, j, nrows, ncols,1)
            feat[cur_idx,15] = np.sum(input_color[i,:])
            feat[cur_idx,16] = np.sum(input_color[:,j])
            feat[cur_idx,17:21] = neighbours(input_color, i, j, nrows, ncols,2)
            feat[cur_idx,21] = np.max(input_color[i,:])
            feat[cur_idx,22] = np.min(input_color[i,:])
            feat[cur_idx,23] = np.max(input_color[:,j])
            feat[cur_idx,24] = np.min(input_color[:,j])
            feat[cur_idx,25:29] = neighbours(input_color, i, j, nrows, ncols,3)
            feat[cur_idx,29] = np.sum(input_color[i-1:i+1,j-1:j+1])
            feat[cur_idx,30] = np.sum(input_color[i-2:i+2,j-2:j+2])
            feat[cur_idx,31] = len(input_color[i-5:i+5,j-5:j+5])
            #feat[cur_idx,32] = len(np.unique(input_color[i+1,:])) if i+1<nrows else -1
            #feat[cur_idx,33] = np.sum(input_color[i-1,:]) if i-1>0 else -1
            #feat[cur_idx,34] = np.sum(input_color[:,j+1]) if j+1<ncols else -1
            #feat[cur_idx,35] = np.sum(input_color[:,j-1]) if j-1>0 else -1     
            cur_idx += 1
        
    return feat

In [9]:
def features(task, mode='train'):
    num_train_pairs = len(task[mode])
    feat, target = [], []
    
    global local_neighb
    for task_num in range(num_train_pairs):
        input_color = np.array(task[mode][task_num]['input'])
        target_color = task[mode][task_num]['output']
        nrows, ncols = len(task[mode][task_num]['input']), len(task[mode][task_num]['input'][0])

        target_rows, target_cols = len(task[mode][task_num]['output']), len(task[mode][task_num]['output'][0])
        
        if (target_rows!=nrows) or (target_cols!=ncols):
            return None, None, 1

        imsize = nrows*ncols
        offset = imsize*task_num*3 #since we are using three types of aug
        feat.extend(make_features(input_color, nfeat))
        target.extend(np.array(target_color).reshape(-1,))
            
    return np.array(feat), np.array(target), 0

def new_features(task):
    num_train_pairs = len(task)
    feat, target = [], []
    
    global local_neighb
    for task_num in range(num_train_pairs):
        input_color = np.array(task[task_num]['input'])
        target_color = task[task_num]['output']
        nrows, ncols = len(task[task_num]['input']), len(task[task_num]['input'][0])

        target_rows, target_cols = len(task[task_num]['output']), len(task[task_num]['output'][0])
        
        if (target_rows!=nrows) or (target_cols!=ncols):
            return None, None, 1

        imsize = nrows*ncols
        offset = imsize*task_num*3 #since we are using three types of aug
        feat.extend(make_features(input_color, nfeat))  
        target.extend(np.array(target_color).reshape(-1,))
            
    return np.array(feat), np.array(target), 0

# data augmentation

In [10]:
def padding(tasks):
    ans = []
    for task in tasks:
        tmp = {"input": 0, "output": 0}
        tmp["input"] = task["input"]
        saigen = np.zeros(np.array(task["input"]).shape)
        cur_h, cur_w = np.array(task["output"]).shape
        saigen[:cur_h, :cur_w] = np.array(task["output"])
        tmp["output"] = saigen.astype(int).tolist()
        ans.append(tmp)
    return ans, cur_h, cur_w

def data_aug(tasks):
    ans = []
    for i in tasks:
        ans.append(i)
    
    for i in range(len(tasks)):
        tmp1 = {'input': 0, 'output': 0}
        tmp1["input"], tmp1["output"] = np.fliplr(tasks[i]["input"]).tolist(), np.fliplr(tasks[i]["output"]).tolist()
        tmp2 = {'input': 0, 'output': 0}
        tmp2["input"], tmp2["output"] = np.flipud(tasks[i]["input"]).tolist(), np.flipud(tasks[i]["output"]).tolist()
        tmp3 = {'input': 0, 'output': 0}
        tmp3["input"], tmp3["output"] = np.rot90(tasks[i]["input"]).tolist(), np.rot90(tasks[i]["output"]).tolist()
        tmp4 = {'input': 0, 'output': 0}
        tmp4["input"], tmp4["output"] = np.rot90(np.fliplr(tasks[i]["input"]),1).tolist(), np.rot90(np.fliplr(tasks[i]["output"]),1).tolist()
        tmp5 = {'input': 0, 'output': 0}
        tmp5["input"], tmp5["output"] = np.rot90(np.fliplr(tasks[i]["input"]),2).tolist(), np.rot90(np.fliplr(tasks[i]["output"]),2).tolist()
        tmp6 = {'input': 0, 'output': 0}
        tmp6["input"], tmp6["output"] = np.rot90(np.fliplr(tasks[i]["input"]),3).tolist(), np.rot90(np.fliplr(tasks[i]["output"]),3).tolist()
        tmp7 = {'input': 0, 'output': 0}
        tmp7["input"], tmp7["output"] = np.rot90(np.flipud(tasks[i]["input"]),1).tolist(), np.rot90(np.flipud(tasks[i]["output"]),1).tolist()
        tmp8 = {'input': 0, 'output': 0}
        tmp8["input"], tmp8["output"] = np.rot90(np.flipud(tasks[i]["input"]),2).tolist(), np.rot90(np.flipud(tasks[i]["output"]),2).tolist()
        tmp9 = {'input': 0, 'output': 0}
        tmp9["input"], tmp9["output"] = np.rot90(np.flipud(tasks[i]["input"]),3).tolist(), np.rot90(np.flipud(tasks[i]["output"]),3).tolist()
        tmp10 = {'input': 0, 'output': 0}
        tmp10["input"], tmp10["output"] = np.fliplr(np.flipud(tasks[i]["input"])).tolist(), np.fliplr(np.flipud(tasks[i]["output"])).tolist()
        #tmp11 = {'input': 0, 'output': 0}
        #tmp11["input"], tmp11["output"] = np.transpose(tasks[i]["input"]).tolist(), np.transpose(tasks[i]["output"]).tolist()
        ans.append(tmp1)
        ans.append(tmp2)
        ans.append(tmp3)
        ans.append(tmp4)
        ans.append(tmp5)
        ans.append(tmp6)
        ans.append(tmp7)
        ans.append(tmp8)
        ans.append(tmp9)
        ans.append(tmp10)
    return ans

def data_aug2(a):
    new_data = []
    for job in a:
        new_data.append(job)
    for job in a:
        color = [i for i in np.unique(np.array(job["input"])) if i != 0]
        color_out = [i for i in np.unique(np.array(job["output"])) if i != 0]
        if set(color) == set(color_out):
            color_pos = []
            color_posout = []
            for i in color: # for input
                tmp = np.argwhere(np.array(job["input"])==i).tolist()
                color_pos.append(tmp)
            for i in color: # for output
                tmp = np.argwhere(np.array(job["output"])==i).tolist()
                color_posout.append(tmp)
            ind = [j for j in range(len(color))]
            for i,ele in enumerate(permutations(ind)):
                if i != 0:
                    tmp1 = np.copy(job["input"])
                    for c in range(len(ele)):
                        for pos in color_pos[ele[c]]:
                            tmp1[pos[0],pos[1]] = color[c]
                    tmp2 = np.copy(job["output"])
                    for c in range(len(ele)):
                        for pos in color_posout[ele[c]]:
                            tmp2[pos[0],pos[1]] = color[c]
                    if len(new_data) >50:
                        break
                    new_data.append({"input":tmp1.tolist(), "output":tmp2.tolist()})
        else:
            return a
    return new_data

# ------------------------------------------------------------------------
# ------------------------------------------------------------------------

def getiorc(pair):
    inp = pair["input"]
    return pair["input"],pair["output"],len(inp),len(inp[0])

def getBkgColor(task_json):
    color_dict = defaultdict(int)
    
    for pair in task_json:
        inp,oup,r,c = getiorc(pair)
        for i in range(r):
            for j in range(c):
                color_dict[inp[i][j]]+=1
    color = -1
    max_count = 0
    for col,cnt in color_dict.items():
        if(cnt > max_count):
            color = col
            max_count = cnt
    return color

def replace(inp,uni,perm):
    # uni = '234' perm = ['5','7','9']
    r_map = { int(c):int(s) for c,s in zip(uni,perm)}
    r,c = len(inp),len(inp[0])
    rp = np.array(inp).tolist()
    for i in range(r):
        for j in range(c):
            if(rp[i][j] in r_map):
                rp[i][j] = r_map[rp[i][j]]
    return rp

def augment(inp,oup,bl_cols):
    cols = "0123456789"
    npr_map = [1,9,72,3024,15120,60480,181440,362880,362880]
    uni = "".join([str(x) for x in np.unique(inp).tolist()])
    for c in bl_cols:
        cols=cols.replace(str(c),"")
        uni=uni.replace(str(c),"")

    exp_size = len(inp)*len(inp[0])*npr_map[len(uni)]
    
    mod = floor(exp_size/120000)
    mod = 1 if mod==0 else mod
    
    #print(exp_size,mod,len(uni))
    result = []
    count = 0
    for comb in combinations(cols,len(uni)):
        for perm in permutations(comb):
            count+=1
            if(count % mod == 0):
                result.append({"input":replace(inp,uni,perm),"output":replace(oup,uni,perm)})
    return result

def get_bl_cols(task_json):
    result = []
    bkg_col = getBkgColor(task_json);
    result.append(bkg_col)
    # num_input,input_cnt,num_output,output_cnt
    met_map = {}
    for i in range(10):
        met_map[i] = [0,0,0,0]
        
    total_ex = 0
    for pair in task_json:
        inp,oup=pair["input"],pair["output"]
        u,uc = np.unique(inp, return_counts=True)
        inp_cnt_map = dict(zip(u,uc))
        u,uc = np.unique(oup, return_counts=True)
        oup_cnt_map = dict(zip(u,uc))
        
        for col,cnt in inp_cnt_map.items():
            met_map[col][0] = met_map[col][0] + 1
            met_map[col][1] = met_map[col][1] + cnt
        for col,cnt in oup_cnt_map.items():
            met_map[col][2] = met_map[col][2] + 1
            met_map[col][3] = met_map[col][3] + cnt
        total_ex+=1
        
    for col,met in met_map.items():
        num_input,input_cnt,num_output,output_cnt = met
        if(num_input == total_ex or num_output == total_ex):
            result.append(col)
        elif(num_input == 0 and num_output > 0):
            result.append(col)
    
    result = np.unique(result).tolist()
    if(len(result) == 10):
        result.append(bkg_col)
    return np.unique(result).tolist()

def data_aug3(task_json):
    ans = []
    for i in task_json:
        ans.append(i)
        
    bl_cols = get_bl_cols(task_json)
    for pair in task_json:
        inp,oup=pair["input"],pair["output"]
        augs = augment(inp,oup,bl_cols)
        for i in augs:
            ans.append(i)
    return ans

# modelling

In [11]:
nfeat = 32
local_neighb = 5
def modelling(mode):
    print(mode)
    count = 0
    reproduction = 0
    sample_sub = pd.read_csv(data_path/'sample_submission.csv')
    sample_sub = sample_sub.set_index('output_id')
    
    valid_scores = {}
    model_accuracies = {'ens': []}
    pred_taskids = []
    
    if mode=='eval':
        task_path = evaluation_path
        df = evaluation_descriptive_df
    elif mode=='train':
        task_path = training_path
        df = training_descriptive_df
    elif mode=='test':
        task_path = test_path
        df = test_descriptive_df
    all_task_ids = sorted(os.listdir(task_path))
    # training ----------
    for task_id in all_task_ids:
        class_num = df[df.task_name==task_id]["class"].values[0]
        task_file = str(task_path / task_id)
        with open(task_file, 'r') as f:
            task = json.load(f)
        
        if mode != "test":
            task = preprocess(task, task_id)

        if class_num == 1 or class_num == 3 or class_num == 5 or class_num == 6:
            _, _, not_valid = features(task)
            if not_valid:
                print(str(class_num) + ' ignoring task', task_file)
                count += 1
                continue
            task["train"] = data_aug3(task["train"])
            feat, target, _ = features(task)
            
        elif class_num == 2 or class_num == 4:
            _, _, not_valid = features(task)
            if not_valid:
                print(str(class_num) + ' ignoring task', task_file)
                count += 1
                continue
            task["train"] = data_aug(task["train"])
            feat, target, _ = features(task)
            
        elif class_num == 7:
            task["train"], out_row, out_col = padding(task["train"])
            task["train"] = data_aug(task["train"])
            feat, target, _ = features(task)
            
        else:
            feat, target, not_valid = features(task)
            if not_valid:
                print(str(class_num) + ' ignoring task', task_file)
                count += 1
                continue
         
        model = XGBClassifier(n_estimators=50, max_depth = 5, num_leaves=10, learning_rate=0.1, n_jobs=-1)        
        model.fit(feat, target, verbose=0)
    # training on input pairs is done ----------
    
    # reprediction -------
        num_train_pairs = len(task["train"])
        count_rep = 0
        new_tasks = []
        for task_num in range(num_train_pairs):
            nrows, ncols = np.array(task['train'][task_num]['input']).shape[0], np.array(task['train'][task_num]['input']).shape[1]
            tmp_feat = feat[count_rep:count_rep+nrows*ncols,:]
            tmp_target = task['train'][task_num]['output']
            train_preds = model.predict(tmp_feat).reshape(nrows,ncols).tolist()
            tmp_dict = {"input": train_preds, "output": tmp_target}
            new_tasks.append(tmp_dict)
            count_rep += nrows * ncols
        new_feat, new_target, _ = new_features(new_tasks)
        model2 = XGBClassifier(n_estimators=50, max_depth = 5, num_leaves=10, learning_rate=0.1, n_jobs=-1)
        model2.fit(new_feat, new_target, verbose=0)
    # -------
    
    # test predictions begins here
        num_test_pairs = len(task['test'])
        for task_num in range(num_test_pairs):
            cur_idx = 0
            input_color = np.array(task['test'][task_num]['input'])
            nrows, ncols = len(task['test'][task_num]['input']), len(task['test'][task_num]['input'][0])
            feat = make_features(input_color, nfeat)
            preds = model.predict(feat).reshape(nrows,ncols)
            new_test_feat = make_features(preds, nfeat) #**
            new_test_preds = model2.predict(new_test_feat).reshape(nrows,ncols) #**
            if class_num == 7:
                preds_7 = np.array(preds)[:out_row, :out_col]
                new_test_preds_7 = np.array(new_test_preds)[:out_row, :out_col]
            if (mode=='train') or (mode=='eval'):
                if class_num == 7:                    
                    ens_acc = (np.array(task['test'][task_num]['output'])==preds_7).sum()/(out_row*out_col)
                    new_ens_acc = (np.array(task['test'][task_num]['output'])==new_test_preds_7).sum()/(out_row*out_col)
                else:
                    ens_acc = (np.array(task['test'][task_num]['output'])==preds).sum()/(nrows*ncols)
                    new_ens_acc = (np.array(task['test'][task_num]['output'])==new_test_preds).sum()/(nrows*ncols)
                model_accuracies['ens'].append(ens_acc)
                pred_taskids.append(f'{task_id[:-5]}_{task_num}')
                print(str(class_num) + ", " + str(task_id) + ' ensemble accuracy',ens_acc)
                if ens_acc < 1 and new_ens_acc == 1:
                    print("*******************reprediction success**********************")
                    reproduction += 1
            else:
                if class_num == 7:
                    preds_7 = preds7.astype(int).tolist()
                    new_test_preds7 = new_test_preds_7.astype(int).tolist()
                    sample_sub.loc[f'{task_id[:-5]}_{task_num}','output'] = flattener(preds_7)+" "+flattener(new_test_preds_7) #flattener(preds)
                else:
                    preds = preds.astype(int).tolist()
                    new_test_preds = new_test_preds.astype(int).tolist()
                    sample_sub.loc[f'{task_id[:-5]}_{task_num}','output'] = flattener(preds)+" "+flattener(new_test_preds) #flattener(preds)
    print(str(count) + " tasks were ignored.")
    print(str(reproduction) + " tasks were corrected")
    return sample_sub, model_accuracies, pred_taskids

_, train_xgb_accuracies, train_ids= modelling('train')
_, eval_xgb_accuracies, eval_ids  = modelling('eval')

train
8 ignoring task /kaggle/input/abstraction-and-reasoning-challenge/training/007bbfb7.json
2, 00d62c1b.json ensemble accuracy 0.94
8 ignoring task /kaggle/input/abstraction-and-reasoning-challenge/training/017c7c7b.json
6, 025d127b.json ensemble accuracy 1.0
6, 045e512c.json ensemble accuracy 0.9070294784580499
7, 0520fde7.json ensemble accuracy 0.6666666666666666
4, 05269061.json ensemble accuracy 0.673469387755102
3, 05f2a901.json ensemble accuracy 0.9
6, 06df4c85.json ensemble accuracy 0.9289940828402367
2, 08ed6ac7.json ensemble accuracy 0.8148148148148148
1, 09629e4f.json ensemble accuracy 0.7107438016528925
6, 0962bcdd.json ensemble accuracy 0.9444444444444444
*******************reprediction success**********************
6, 0a938d79.json ensemble accuracy 0.8653198653198653
8 ignoring task /kaggle/input/abstraction-and-reasoning-challenge/training/0b148d64.json
5, 0ca9ddb6.json ensemble accuracy 0.9382716049382716
6, 0d3d703e.json ensemble accuracy 0.0
4, 0dfd9992.json ensemb

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:100: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


AttributeError: 'bool' object has no attribute 'sum'

In [12]:
df = pd.DataFrame(train_xgb_accuracies, index=train_ids)
for c in df.columns:
    print(f'for {c} no. of complete training tasks is', (df.loc[:, c]==1).sum())
    print(df[df.loc[:, c]==1].index)
    print(f'for {c} no. of complete training tasks is', (df.loc[:, c]>0.9).sum())

df = pd.DataFrame(eval_xgb_accuracies, index=eval_ids)
for c in df.columns:
    print(f'for {c} no. of complete evaluation tasks is', (df.loc[:, c]==1).sum())
    print(df[df.loc[:, c]==1].index)
    print(f'for {c} no. of complete evaluation tasks is', (df.loc[:, c]>0.9).sum())

NameError: name 'train_xgb_accuracies' is not defined

# Prediction

In [13]:
test_xgb, _, _ = modelling('test')
test_xgb.to_csv("submission.csv")

test
8 ignoring task /kaggle/input/abstraction-and-reasoning-challenge/test/00576224.json
8 ignoring task /kaggle/input/abstraction-and-reasoning-challenge/test/0692e18c.json
8 ignoring task /kaggle/input/abstraction-and-reasoning-challenge/test/0934a4d8.json
8 ignoring task /kaggle/input/abstraction-and-reasoning-challenge/test/0a1d4ef5.json


NameError: name 'preds7' is not defined

In [14]:
#final_sub = test_xgb.reset_index()
#final_sub = final_sub.sort_values(by="output_id")

#test_lgb = test_lgb.sort_values(by="output_id")
#test_cat = test_cat.sort_values(by="output_id")
#out1 = final_sub["output"].astype(str).values
#out2 = test_lgb["output"].astype(str).values
#out3 = test_cat["output"].astype(str).values

#merge_output = []
#for o1, o2, o3 in zip(out1, out2, out3):
#    o = o1.strip().split(" ")[:1] + o2.strip().split(" ")[:1] + o2.strip().split(" ")[:1]
#    o = " ".join(o[:3])
#    merge_output.append(o)
#final_sub["output"] = merge_output
#final_sub["output"] = final_sub["output"].astype(str)
#final_sub.to_csv("submission.csv", index=False)
#final_sub.head()